In [23]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re


In [49]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Collect penetration dates from the forum

In [ ]:
#Load data:

pages_pd = pd.read_csv('scraped_product_pages_extracted.csv', sep=";", index_col = "Unnamed: 0") 

For the ones with a github page:

In [ ]:
pages_penetrated = pages[(pages["nonzero_price"]) & (pages["open_issues"].isnull()==False)]

In [ ]:
pages_penetrated["search_query"]=pages_penetrated["githublink"].str.split('/').str[4]

In [ ]:
pages_penetrated_dict = pages_penetrated[["url", "search_query", "main_developer_username", "launch_date"]].to_dict(orient="records")

In [ ]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [ ]:
for page in pages_penetrated_dict:
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation")
    time.sleep(2)
    driver.find_element(by=By.CLASS_NAME, value = "quickSearch-ResultCloser").click()
    time.sleep(1)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)

        if results:
            earliest_post = min(results, key=lambda x:x['post_number'])
            page["penetration_datestamp"] = earliest_post["datestamp"]
        else: 
            page["penetration_datestamp"] = np.nan
driver.quit()   

In [ ]:
with open('pages_github_penetrated_datestamps.json', 'w') as fp:
    json.dump(pages_penetrated_dict, fp)

For all non-zero priced products:

In [12]:
# Load clean data:
pages = pd.read_csv('data/safe_obs_dataset.csv', sep=";", index_col = "Unnamed: 0") 

In [36]:
pages

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_1,contrlink_2,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,"May 2, 2022","\nThe compass does not point to the north, but...",['https://play.google.com/store/apps/details?i...,Baiumka,/m/baiumka.12120441/,New member,"May 2, 2022",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True
7,/t/app-2-2-root-wifi-reaver-gui-for-android.24...,[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android,"Sep 24, 2013",\n\n\n\n\n\nReaver for Android v1.30\nReaver-W...,['https://bcmon.googlecode.com/files/bcmon.apk...,SOEDI,/m/soedi.4096618/,Senior Member,"May 14, 2011",154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
8,/t/app-11-dual-camera-video-recorder.4447581/,[APP][11+] Dual Camera Video Recorder,Today at 8:08 AM,\nCapture video from rear and front cameras at...,['https://play.google.com/store/apps/details?i...,User848,/m/user848.5214037/,Senior Member,"Apr 9, 2013",78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38406,/t/free-apps-packet-good-apps-packet.678403/,[FREE APPS PACKET] Good apps packet.,"May 9, 2010",\neeehh... sory delete this topic\n \n,[],Cypek,/m/cypek.2556290/,New member,"May 6, 2010",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38415,/t/full-video-chatt-app-like-skype.675381/,Full Video Chatt App like skype,"May 4, 2010",\nim deploying soon and was wondering if anyon...,[],pyrator,/m/pyrator.1603376/,Senior Member,"Feb 28, 2009",307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38420,/t/help-me-in-getting-this-app.674932/,help me in getting this app,"May 3, 2010","\nHi all,\n\ni used this(dont remember the nam...",[],chowdarygm,/m/chowdarygm.2004328/,Senior Member,"Sep 4, 2009",872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38435,/t/market-as-separate-app.672499/,Market as separate app,"Apr 29, 2010","\nHey people, I rebooted my phone this morning...",[],fuz1987,/m/fuz1987.2336665/,Member,"Feb 3, 2010",20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True


In [13]:
pages_nonzero_dict = pages[pages["nonzero_price"]][["url","main_developer_username", "launch_date"]].to_dict(orient="records")

In [9]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [17]:
link_n = pages.loc[2, "url"]

In [19]:
driver.get("https://forum.xda-developers.com" + link_n)

In [ ]:
for page in pages_nonzero_dict[747:]:
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "quickSearch-ResultCloser"))).click()
    time.sleep(2)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)
    time.sleep(2)
    while True:
        next_page_btn = driver.find_elements(by=By.LINK_TEXT, value = "Next")
        if len(next_page_btn) < 1:
           
            break
        else:
            next_page = driver.find_element(by=By.CLASS_NAME, value = "pageNav-jump.pageNav-jump--next").get_attribute("href")
            driver.get(next_page)
            time.sleep(2)
            for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
                title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
                if re.search("Post", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                    post_number = int(re.search(r"[\d,]+", post_number).group(0))
                    mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
                    results.append(mention)
    if results:
        earliest_post = min(results, key=lambda x:x['post_number'])
        page["penetration_datestamp"] = earliest_post["datestamp"]
    else: 
        page["penetration_datestamp"] = np.nan
driver.quit()    


In [58]:
page["url"]

'/t/app-please-read-free-download-superbeam-pro-unlocker.2771325/'

In [52]:
scraped = pd.DataFrame(pages_nonzero_dict)

In [61]:
scraped.loc[scraped["url"] == page["url"],"url"]

745    /t/app-please-read-free-download-superbeam-pro...
Name: url, dtype: object

In [41]:
left

[4,
 5,
 6,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 19

In [29]:
next_page = driver.find_element(by=By.CLASS_NAME, value = "pageNav-jump.pageNav-jump--next").get_attribute("href")

'https://forum.xda-developers.com/search/42083685/?page=3&q=pro+%7Cpurchase%7Cdonation&t=post&c[thread]=3233012&c[users]=M66B&o=relevance'

In [ ]:
driver.get()

In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(2)
driver.find_element(by=By.LINK_TEXT, value = "Next").click()


In [ ]:
driver.find_element(by=By.LINK_TEXT, value = "Next").click()

In [ ]:
driver.quit()

# Collect mentions of product in other forums after the date